# Heidi Luna PHW3: Chapter 4 and 5 Propositions and Exercises

# Chapter 4: Subqueries - Propositions

# Propositions and Functional Specification: Propositon 1

## 1. Query Proposition

The purpose of this query is to retrieve a list of the top 9 employees based on the total number of orders they have processed. It also calculates the unique customers served by each employee, providing insights into employee performance in the context of customer engagement.

## 2. Functional Specification

### Objective
The query aims to analyze employee productivity by identifying the employees who handle the most orders and the number of unique customers they serve.

### Inputs
- **Database Tables**:
  - `HumanResources.Employee`: Contains employee details, including `EmployeeID`, `EmployeeFirstName`, and `EmployeeLastName`.
  - `Sales.[Order]`: Contains order details, including `OrderID` and `EmployeeID`.
  - `Sales.Customer`: Contains customer information, including `CustomerID`.

### Outputs
- **Columns**:
  - `EmployeeID`: Unique identifier for each employee.
  - `EmployeeName`: Concatenation of the employee's first and last names.
  - `TotalOrders`: The total number of orders processed by the employee.
  - `UniqueCustomers`: The count of distinct customers served by the employee.

### Query Logic

1. **Joins**:
   - The query joins the `HumanResources.Employee` table with the `Sales.[Order]` table on the `EmployeeID` to associate employees with their orders.

2. **COUNT Function**:
   - The query counts the total number of orders (`OrderID`) for each employee using the `COUNT` function.

3. **Subquery**:
   - A subquery calculates the number of distinct customers (`CustomerID`) associated with the orders processed by each employee. This subquery joins the `Sales.Customer` and `Sales.[Order]` tables, filtering based on the `EmployeeID`.

4. **Grouping**:
   - The results are grouped by `EmployeeID`, `EmployeeFirstName`, and `EmployeeLastName` to aggregate total orders and unique customers correctly.

5. **Ordering and Limiting**:
   - The results are ordered by `TotalOrders` in descending order to highlight the most productive employees.
   - The `TOP 25` clause limits the results to the top 9 employees based on the number of orders.

### Sample SQL Query

```sql
USE Northwinds2022TSQLV7;

SELECT TOP 9
       E.EmployeeID,
       E.FirstName + ' ' + E.LastName AS EmployeeName,
       COUNT(O.OrderID) AS TotalOrders,
       (SELECT COUNT(DISTINCT C.CustomerID)
        FROM Sales.Customer C
        JOIN Sales.[Order] O2 ON C.CustomerID = O2.CustomerID
        WHERE O2.EmployeeID = E.EmployeeID) AS UniqueCustomers
FROM HumanResources.Employee E
JOIN Sales.[Order] O ON E.EmployeeID = O.EmployeeID
GROUP BY E.EmployeeID, E.FirstName, E.LastName
ORDER BY TotalOrders DESC;


In [18]:
USE Northwinds2022TSQLV7;

SELECT TOP 25
       E.EmployeeID,
       E.EmployeeFirstName + ' ' + E.EmployeeLastName AS EmployeeName,
       COUNT(O.OrderID) AS TotalOrders,
       (SELECT COUNT(DISTINCT C.CustomerID)
        FROM Sales.Customer C
        JOIN Sales.[Order] O2 ON C.CustomerID = O2.CustomerID
        WHERE O2.EmployeeID = E.EmployeeID) AS UniqueCustomers
FROM HumanResources.Employee E
JOIN Sales.[Order] O ON E.EmployeeID = O.EmployeeID
GROUP BY E.EmployeeID, E.EmployeeFirstName, E.EmployeeLastName
ORDER BY TotalOrders DESC;

(9 rows affected)

Total execution time: 00:00:00.055

EmployeeID,EmployeeName,TotalOrders,UniqueCustomers
4,Yael Peled,156,75
3,Judy Lew,127,63
1,Sara Davis,123,65
8,Maria Cameron,104,56
2,Don Funk,96,59
7,Russell King,72,45
6,Paul Suurs,67,43
9,Patricia Doyle,43,29
5,Sven Mortensen,42,29


# Propositions and Functional Specification : Proposition 2

## 1. Query Proposition

This query is designed to retrieve customers from the **Northwinds2022TSQLV7** database who have ordered more than five distinct products. By using subqueries, joins, and aggregation functions, the query calculates the distinct count of products for each customer. This information is essential for identifying customers with diverse purchasing behaviors, which can aid in marketing and inventory management strategies.

## 2. Functional Specification

### Objective
Retrieve customers who have placed orders for more than five distinct products, providing insights into customer engagement and product variety.

### Inputs
- **Database Tables**:
  - `Sales.Customer`: Contains customer information (`CustomerID`, `CustomerCompanyName`).
  - `Sales.[Order]`: Contains order information including `OrderID` and the `CustomerID` who placed each order.
  - `Sales.OrderDetail`: Contains details about each order item (`OrderID`, `ProductID`).

### Outputs
- **Columns**:
  - `CustomerID`: The unique identifier for each customer.
  - `CustomerCompanyName`: The name of the customer’s company.
  - `DistinctProductCount`: The total number of distinct products ordered by the customer.

### Query Logic

1. **Subquery (DistinctProductCount Calculation)**:
   - Joins the `Sales.OrderDetail` and `Sales.[Order]` tables to count distinct products ordered by each customer.
   - Filters the results to count only the distinct `ProductID` for each customer's orders.

2. **Main Query**:
   - Selects `CustomerID` and `CustomerCompanyName` from the `Sales.Customer` table.
   - Uses the subquery to filter out customers who have ordered more than five distinct products.
   - Orders the results by `DistinctProductCount` in descending order to prioritize customers with the most diverse orders.

### Performance Considerations
- **Subquery Execution**: The subquery calculates the distinct product count and may lead to performance issues with large datasets, but proper indexing on `ProductID` and `OrderID` can enhance performance.
- **Join Operations**: The query involves joining across multiple tables, which can be optimized with indexing on `CustomerID` in the `Sales.[Order]` and `Sales.Customer` tables to reduce lookup times.



In [9]:
USE Northwinds2022TSQLV7;

SELECT C.CustomerID, C.CustomerCompanyName, 
       (SELECT COUNT(DISTINCT OD.ProductID) 
        FROM Sales.OrderDetail OD
        JOIN Sales.[Order] O ON OD.OrderID = O.OrderID
        WHERE O.CustomerID = C.CustomerID) AS DistinctProductCount
FROM Sales.Customer C
WHERE (SELECT COUNT(DISTINCT OD.ProductID) 
       FROM Sales.OrderDetail OD
       JOIN Sales.[Order] O ON OD.OrderID = O.OrderID
       WHERE O.CustomerID = C.CustomerID) > 5
ORDER BY DistinctProductCount DESC;

(85 rows affected)

Total execution time: 00:00:00.044

CustomerID,CustomerCompanyName,DistinctProductCount
20,Customer THHDP,56
71,Customer LCOUJ,53
63,Customer IRRVL,49
65,Customer NYUHS,45
5,Customer HGVLZ,37
37,Customer FRXZL,36
9,Customer RTXGC,34
35,Customer UMTLM,33
25,Customer AZJED,33
62,Customer WFIZJ,33


# Propositions and Functional Specification : Proposition 3

## 1. Query Proposition

This query is designed to retrieve the top 10 customers from the **Northwinds2022TSQLV7**  database based on the Total Revenue Generated of their orders. The query makes use of subqueries, joins, and aggregation functions to calculate the total order value for each customer and then selects the customers with the highest total values. It provides insights into which customers have generated the most revenue.

## 2. Functional Specification

### Objective
Retrieve the top 10 customers who have placed the highest value of orders, measured by the sum of the total price of their purchased products.

### Inputs
- **Database Tables**:
  - `Customers`: Contains customer information (`CustomerID`, `CompanyName`).
  - `Orders`: Contains order details including `OrderID`, and the `CustomerID` who placed each order.
  - `Order Details`: Contains specific product information for each order (`OrderID`, `UnitPrice`, `Quantity`).

### Outputs
- **Columns**:
  - `CustomerID`: The unique identifier for the customer.
  - `CompanyName`: The name of the customer’s company.
  - `TotalOrderValue`: The total value of all orders placed by the customer, calculated by summing the product of `UnitPrice` and `Quantity` for each order.

### Query Logic

1. **Subquery (CustomerOrderValues)**:
   - Joins the `Orders` and `Order Details` tables to calculate the total value of each order by multiplying the `UnitPrice` by `Quantity` for each line item in the `Order Details` table.
   - Groups the result by `CustomerID` to get the total order value for each customer.
   
2. **Main Query**:
   - Joins the results from the subquery with the `Customers` table on `CustomerID` to retrieve the `CustomerID` and `CompanyName`.
   - Orders the results by `TotalOrderValue` in descending order to prioritize customers with the highest order value.
   - Limits the results to the top 10 customers using `SELECT TOP 10`.

### Performance Considerations
- **Aggregation**: The `SUM(OD.UnitPrice * OD.Quantity)` in the subquery might become expensive for large datasets, but this can be optimized with appropriate indexing.
- **Join Operations**: Joining across multiple tables (`Orders` and `Order Details`) ensures the calculation of total order value for each customer, but indexing on `OrderID` and `CustomerID` can improve performance.


In [12]:
USE Northwinds2022TSQLV7;

SELECT TOP 10 C.CustomerID, C.CustomerCompanyName, CustomerOrderValues.TotalOrderValue
FROM Sales.Customer C
JOIN (
    SELECT O.CustomerID, SUM(OD.UnitPrice * OD.Quantity) AS TotalOrderValue
    FROM Sales.[Order] O
    JOIN Sales.OrderDetail OD ON O.OrderID = OD.OrderID
    GROUP BY O.CustomerID
) AS CustomerOrderValues
ON C.CustomerID = CustomerOrderValues.CustomerID
ORDER BY CustomerOrderValues.TotalOrderValue DESC;

(10 rows affected)

Total execution time: 00:00:00.049

CustomerID,CustomerCompanyName,TotalOrderValue
63,Customer IRRVL,117483.39
71,Customer LCOUJ,115673.39
20,Customer THHDP,113236.68
37,Customer FRXZL,57317.39
65,Customer NYUHS,52245.90
34,Customer IBVRG,34101.15
24,Customer CYZTN,32555.55
51,Customer PVDZC,32203.90
39,Customer GLLAG,31745.75
62,Customer WFIZJ,30226.10


# Propositions and Functional Specification: Proposition 4

## 1. Query Proposition

This query aims to retrieve detailed information about employees and the customers they serve from the **Northwinds2022TSQLV7** database. By leveraging subqueries, joins, and aggregation, it calculates the total number of orders placed by each customer assigned to an employee. This information is valuable for assessing employee performance and understanding customer engagement.

## 2. Functional Specification

### Objective
To obtain a list of employees along with their customers and the total number of orders placed by each customer that the employee has served.

### Inputs
- **Database Tables**:
  - **HumanResources.Employee**: Contains employee information, including:
    - `EmployeeID`: Unique identifier for each employee.
    - `EmployeeFirstName`: First name of the employee.
    - `EmployeeLastName`: Last name of the employee.
  - **Sales.[Order]**: Contains order details, including:
    - `OrderID`: Unique identifier for each order.
    - `CustomerID`: Identifier for the customer who placed the order.
    - `EmployeeID`: Identifier for the employee who processed the order.
  - **Sales.Customer**: Contains customer information, including:
    - `CustomerID`: Unique identifier for each customer.
    - `CustomerCompanyName`: Name of the customer's company.

### Outputs
- **Columns**:
  - `EmployeeID`: The unique identifier for the employee.
  - `EmployeeName`: The full name of the employee (concatenated first and last names).
  - `CustomerID`: The unique identifier for the customer.
  - `CustomerCompanyName`: The name of the customer's company.
  - `TotalOrders`: The total number of orders placed by the customer served by the employee.

### Query Logic

1. **Main Query**:
   - Selects `EmployeeID`, concatenated `EmployeeName`, `CustomerID`, and `CustomerCompanyName` from the joined tables.
   
2. **Subquery (TotalOrders)**:
   - A subquery counts the total number of orders for each customer served by the specific employee.
   - The subquery checks for orders where the `CustomerID` matches the customer associated with the employee.

3. **Joins**:
   - The query includes:
     - `JOIN` between `HumanResources.Employee` and `Sales.[Order]` on `EmployeeID`.
     - `JOIN` between `Sales.[Order]` and `Sales.Customer` on `CustomerID`.

4. **Grouping**:
   - The `GROUP BY` clause ensures the results are grouped by employee and customer details to facilitate accurate aggregation of total orders.

5. **Ordering**:
   - The results are ordered by `EmployeeID` and `TotalOrders` in descending order to highlight employees with the most orders.

### Performance Considerations
- **Aggregation**: The `COUNT(O.OrderID)` subquery may have performance implications on large datasets, but can be optimized with proper indexing on `OrderID` and `CustomerID`.
- **Join Operations**: The joins ensure accurate data retrieval from multiple tables. Indexing on `EmployeeID` and `CustomerID` can enhance performance.


In [14]:
USE Northwinds2022TSQLV7;

SELECT E.EmployeeID, 
       E.EmployeeFirstName + ' ' + E.EmployeeLastName AS EmployeeName,
       C.CustomerID,
       C.CustomerCompanyName,
       (SELECT COUNT(O.OrderID) 
        FROM Sales.[Order] O 
        WHERE O.CustomerID = C.CustomerID 
          AND O.EmployeeID = E.EmployeeID) AS TotalOrders
FROM HumanResources.Employee E
JOIN Sales.[Order] O ON E.EmployeeID = O.EmployeeID
JOIN Sales.Customer C ON O.CustomerID = C.CustomerID
GROUP BY E.EmployeeID, E.EmployeeFirstName, E.EmployeeLastName, C.CustomerID, C.CustomerCompanyName
ORDER BY E.EmployeeID, TotalOrders DESC;



(464 rows affected)

Total execution time: 00:00:00.053

EmployeeID,EmployeeName,CustomerID,CustomerCompanyName,TotalOrders
1,Sara Davis,71,Customer LCOUJ,6
1,Sara Davis,20,Customer THHDP,5
1,Sara Davis,25,Customer AZJED,5
1,Sara Davis,65,Customer NYUHS,5
1,Sara Davis,63,Customer IRRVL,4
1,Sara Davis,39,Customer GLLAG,4
1,Sara Davis,5,Customer HGVLZ,4
1,Sara Davis,80,Customer VONTK,4
1,Sara Davis,83,Customer ZRNDE,4
1,Sara Davis,4,Customer HFBZG,3


# Propositions and Functional Specification: Proposition 5

## 1. Query Proposition

The purpose of this query is to retrieve detailed information about suppliers and the products they offer from the **Northwinds2022TSQLV7** database. The query aims to analyze the relationship between suppliers and their products by counting the total number of products per supplier and calculating the total value of non-discontinued products supplied.

## 2. Functional Specification

### Objective
The query provides a list of suppliers along with the total number of products they supply and the total value of those products. This information can be useful for evaluating supplier performance and inventory management.

### Inputs
- **Database Tables**:
  - `Production.Supplier`: Contains supplier details, including `SupplierID` and `SupplierCompanyName`.
  - `Production.Product`: Contains product details, including `ProductID`, `SupplierID`, `UnitPrice`, and `Discontinued` status.

### Outputs
- **Columns**:
  - `SupplierID`: Unique identifier for each supplier.
  - `SupplierCompanyName`: The name of the supplier's company.
  - `TotalProducts`: The count of products associated with each supplier that are not discontinued.
  - `TotalProductValue`: The sum of the `UnitPrice` of all non-discontinued products from each supplier.

### Query Logic

1. **Joins**:
   - The query joins the `Production.Supplier` table with the `Production.Product` table using `SupplierID`.

2. **Subquery**:
   - A subquery calculates the total value of non-discontinued products for each supplier by summing the `UnitPrice` of products where `Discontinued` is equal to 0.

3. **Filtering**:
   - The `WHERE` clause filters the products to include only those that are not discontinued.

4. **Grouping**:
   - The results are grouped by `SupplierID` and `SupplierCompanyName` to aggregate the total count and total value for each supplier.

5. **Ordering**:
   - The results are ordered by `TotalProductValue` in descending order, allowing the identification of suppliers with the highest total product value.

### Use Cases
- **Inventory Management**: Understand the total number of products and their values to manage inventory effectively.
- **Supplier Evaluation**: Identify key suppliers based on the value and quantity of products they provide.

### Example Output
The output of this query will provide a sorted list of suppliers, showing their IDs, names, total number of products, and total value of the non-discontinued products they supply.



In [15]:
USE Northwinds2022TSQLV7;

SELECT S.SupplierID,
       S.SupplierCompanyName,
       COUNT(P.ProductID) AS TotalProducts,
       (SELECT SUM(P2.UnitPrice) 
        FROM Production.Product P2
        WHERE P2.SupplierID = S.SupplierID 
          AND P2.Discontinued = 0) AS TotalProductValue
FROM Production.Supplier S
JOIN Production.Product P ON S.SupplierID = P.SupplierID
WHERE P.Discontinued = 0
GROUP BY S.SupplierID, S.SupplierCompanyName
ORDER BY TotalProductValue DESC;

(28 rows affected)

Total execution time: 00:00:00.045

SupplierID,SupplierCompanyName,TotalProducts,TotalProductValue
18,Supplier LVJUA,2,281.50
7,Supplier GQRCV,4,138.85
8,Supplier BWGYE,4,112.70
3,Supplier STUAZ,3,95.00
11,Supplier ZPYVS,3,89.13
28,Supplier OAVQT,2,89.00
14,Supplier KEREV,3,79.30
29,Supplier OGLRK,2,77.80
20,Supplier CIYNM,2,65.45
2,Supplier VHQZD,3,60.05


# Chapter 5: Table Expressions Propositions

# Propositions and Functional Specification: Proposition 6

## 1. Query Proposition

The purpose of this query is to retrieve the total number of orders processed by each employee from the **Northwinds2022TSQLV7** database. It aims to evaluate employee performance based on the number of orders they have handled.

## 2. Functional Specification

### Objective
The query provides a list of employees along with the total number of orders they have processed. This information can be utilized for performance evaluation and resource allocation in the sales department.

### Inputs
- **Database Tables**:
  - `HumanResources.Employee`: Contains employee details, including `EmployeeID`, `EmployeeFirstName`, and `EmployeeLastName`.
  - `Sales.[Order]`: Contains order details, including `OrderID` and `EmployeeID`.

### Outputs
- **Columns**:
  - `EmployeeID`: Unique identifier for each employee.
  - `EmployeeName`: Concatenation of the employee's first and last names.
  - `TotalOrders`: Count of total orders processed by each employee.

### Query Logic

1. **Common Table Expression (CTE)**:
   - The query uses a CTE named `EmployeeOrderCounts` to calculate the total number of orders for each employee. 
   - It retrieves the `EmployeeID` and the concatenated name of the employee from the `HumanResources.Employee` table and counts the `OrderID` from the `Sales.[Order]` table.

2. **LEFT JOIN**:
   - A `LEFT JOIN` is used between the `HumanResources.Employee` table and the `Sales.[Order]` table to ensure that all employees are included in the results, even if they have not processed any orders.

3. **Grouping**:
   - The results are grouped by `EmployeeID`, `EmployeeFirstName`, and `EmployeeLastName` to aggregate the order counts correctly.

4. **Final Selection**:
   - The final selection retrieves `EmployeeID`, `EmployeeName`, and `TotalOrders` from the CTE.

5. **Ordering**:
   - The results are ordered by `TotalOrders` in descending order, allowing for easy identification of the employees with the highest order counts.

### Use Case
This query can be useful for sales management to identify high-performing employees based on the volume of orders they process. It can also help in understanding workload distribution among employees.


In [20]:
USE Northwinds2022TSQLV7;

WITH EmployeeOrderCounts AS (
    SELECT E.EmployeeID, E.EmployeeFirstName + ' ' + E.EmployeeLastName AS EmployeeName, COUNT(O.OrderID) AS TotalOrders
    FROM  HumanResources.Employee E
    LEFT JOIN Sales.[Order] O ON E.EmployeeID = O.EmployeeID
    GROUP BY E.EmployeeID, E.EmployeeFirstName, E.EmployeeLastName )

SELECT EmployeeID, EmployeeName, TotalOrders
FROM  EmployeeOrderCounts
ORDER BY TotalOrders DESC;

(9 rows affected)

Total execution time: 00:00:00.044

EmployeeID,EmployeeName,TotalOrders
4,Yael Peled,156
3,Judy Lew,127
1,Sara Davis,123
8,Maria Cameron,104
2,Don Funk,96
7,Russell King,72
6,Paul Suurs,67
9,Patricia Doyle,43
5,Sven Mortensen,42


# Propositions and Functional Specification: Proposition 7

## 1. Query Proposition

The objective of this query is to calculate the total order value for each customer in the **Northwinds2022TSQLV7** database. It identifies customers whose total order value exceeds $1000. The results are ordered by the total order value in descending order.

## 2. Functional Specification

### Objective
Retrieve a list of customers along with their total order value, filtered to include only those with a total order value greater than $1000.

### Inputs
- **Database Tables**:
  - `Sales.Customer`: Contains customer information, including `CustomerID` and `CustomerCompanyName`.
  - `Sales.[Order]`: Contains order information, including `OrderID` and `CustomerID`.
  - `Sales.OrderDetail`: Contains details of each order, including `UnitPrice`, `Quantity`, and `OrderID`.

### Outputs
- **Columns**:
  - `CustomerID`: The unique identifier for each customer.
  - `CustomerCompanyName`: The name of the customer's company.
  - `TotalOrderValue`: The cumulative value of all orders placed by the customer.

### Query Logic

1. **Common Table Expression (CTE) - `CustomerOrderTotals`**:
   - Joins the `Sales.Customer`, `Sales.[Order]`, and `Sales.OrderDetail` tables to calculate the total order value for each customer.
   - Uses `SUM(OD.UnitPrice * OD.Quantity)` to compute the total order value, grouping by `CustomerID` and `CustomerCompanyName`.

2. **Main Query**:
   - Selects `CustomerID`, `CustomerCompanyName`, and `TotalOrderValue` from the `CustomerOrderTotals` CTE.
   - Filters results to include only customers with a `TotalOrderValue` greater than $1000.
   - Orders the final results by `TotalOrderValue` in descending order to highlight the highest value customers.

### Example Usage
This query can be used by sales teams or management to identify high-value customers and analyze purchasing behavior, which can inform marketing strategies and customer relationship management.

### Edge Cases
- Customers with no orders will not appear in the result set, as they will not meet the threshold.
- The total order value is calculated based on completed orders only, and any orders not recorded in the `Sales.[Order]` or `Sales.OrderDetail` tables will not be included.



In [24]:
USE Northwinds2022TSQLV7;

WITH CustomerOrderTotals AS (
    SELECT C.CustomerID, C.CustomerCompanyName, SUM(OD.UnitPrice * OD.Quantity) AS TotalOrderValue
    FROM  Sales.Customer C
    JOIN Sales.[Order] O ON C.CustomerID = O.CustomerID
    JOIN Sales.OrderDetail OD ON O.OrderID = OD.OrderID
    GROUP BY  C.CustomerID, C.CustomerCompanyName )

SELECT  CustomerID, CustomerCompanyName, TotalOrderValue
FROM CustomerOrderTotals
WHERE TotalOrderValue > 1000  -- Example filter for total order value
ORDER BY TotalOrderValue DESC;

(84 rows affected)

Total execution time: 00:00:00.051

CustomerID,CustomerCompanyName,TotalOrderValue
63,Customer IRRVL,117483.39
71,Customer LCOUJ,115673.39
20,Customer THHDP,113236.68
37,Customer FRXZL,57317.39
65,Customer NYUHS,52245.90
34,Customer IBVRG,34101.15
24,Customer CYZTN,32555.55
51,Customer PVDZC,32203.90
39,Customer GLLAG,31745.75
62,Customer WFIZJ,30226.10


# Chapter 4 Exercises

## Exercise 3: 

Write a query that returns employees who did not place orders on or after May 1, 2022

<u>Tables Involved:</u> HR.Employees and Sales.Orders


<u>Desired Output:</u>

```
empid          firstname    lastname
-----------    ----------   --------------------
3              Judy         Lew
5              Sven         Mortensen
6              Paul         Suurs
9              Patricia     Doyle

(4 row(s) affected)

```


In [14]:
USE TSQLV4;
SELECT empid, firstname, lastname
FROM HR.Employees
WHERE empid NOT IN
  (SELECT O.empid
   FROM Sales.Orders AS O
   WHERE O.orderdate >= '20220501');

Use Northwinds2022TSQLV7; 

SELECT E.EmployeeId, E.EmployeeFirstName,  E.EmployeeLastName
FROM 
    HumanResources.Employee AS E
WHERE 
    E.EmployeeId NOT IN 
    (SELECT O.EmployeeId
     FROM Sales.[Order] AS O
     WHERE O.OrderDate >= '2022-05-01');

(9 rows affected)

(9 rows affected)

Total execution time: 00:00:00.040

empid,firstname,lastname
1,Sara,Davis
2,Don,Funk
3,Judy,Lew
4,Yael,Peled
5,Sven,Mortensen
6,Paul,Suurs
7,Russell,King
8,Maria,Cameron
9,Patricia,Doyle


EmployeeId,EmployeeFirstName,EmployeeLastName
1,Sara,Davis
2,Don,Funk
3,Judy,Lew
4,Yael,Peled
5,Sven,Mortensen
6,Paul,Suurs
7,Russell,King
8,Maria,Cameron
9,Patricia,Doyle


## Exercise 10: 

Write a query that returns for each order the number of days that passed since the same customer’s previous order. To determine recency among orders, use orderdate as the primary sort element and orderid as the tiebreaker 

<u>Tables Involved:</u> Sales.Order

<u>Desired Output:</u> 
```
custid      orderdate     orderid       diff
----------- ----------    -----------   -----------
1           2021-08-25    10643         NULL
1           2021-10-03    10692         39
1           2021-10-13    10702         10
1           2022-01-15    10835         94
1           2022-03-16    10952         60
1           2022-04-09    11011         24
2           2020-09-18    10308         NULL
2           2021-08-08    10625         324
2           2021-11-28    10759         112
2           2022-03-04    10926         96
...

(830 rows affected)

```

In [5]:
USE TSQLV4;
SELECT custid, orderdate, orderid,
  (SELECT TOP (1) O2.orderdate
  FROM Sales.Orders AS O2
   WHERE O2.custid = O1.custid
     AND (    O2.orderdate = O1.orderdate AND O2.orderid < O1.orderid
           OR O2.orderdate < O1.orderdate )
   ORDER BY O2.orderdate DESC, O2.orderid DESC) AS prevdate
FROM Sales.Orders AS O1
ORDER BY custid, orderdate, orderid;

USE Northwinds2022TSQLV7;
SELECT CustomerId, OrderDate, OrderId,
  (SELECT TOP (1) O2.OrderDate
  FROM Sales.[Order] AS O2
   WHERE O2.CustomerId = O1.CustomerId
     AND (    O2.OrderDate = O1.OrderDate AND O2.OrderId < O1.OrderId
           OR O2.OrderDate < O1.OrderDate )
   ORDER BY O2.OrderDate DESC, O2.OrderId DESC) AS prevdate
FROM Sales.[Order] AS O1
ORDER BY CustomerId, OrderDate, OrderId;

(830 rows affected)

(830 rows affected)

Total execution time: 00:00:00.121

custid,orderdate,orderid,prevdate
1,2015-08-25,10643,NULL
1,2015-10-03,10692,2015-08-25
1,2015-10-13,10702,2015-10-03
1,2016-01-15,10835,2015-10-13
1,2016-03-16,10952,2016-01-15
1,2016-04-09,11011,2016-03-16
2,2014-09-18,10308,NULL
2,2015-08-08,10625,2014-09-18
2,2015-11-28,10759,2015-08-08
2,2016-03-04,10926,2015-11-28


CustomerId,OrderDate,OrderId,prevdate
1,2015-08-25,10643,NULL
1,2015-10-03,10692,2015-08-25
1,2015-10-13,10702,2015-10-03
1,2016-01-15,10835,2015-10-13
1,2016-03-16,10952,2016-01-15
1,2016-04-09,11011,2016-03-16
2,2014-09-18,10308,NULL
2,2015-08-08,10625,2014-09-18
2,2015-11-28,10759,2015-08-08
2,2016-03-04,10926,2015-11-28


# Chspter 5 Exercise

## Exercise 5-1

Create a view that returns the total quantity for each employee and year

<u>Tables involved:</u> Sales.Orders and Sales.OrderDetails

<u>Desired Output:</u> 
```empid       orderyear   qty
----------- ----------- -----------
1           2020        1620
1           2021        3877
1           2022        2315
2           2020        1085
2           2021        2604
2           2022        2366
3           2020        940
3           2021        4436
3           2022        2476
4           2020        2212
4           2021        5273
4           2022        2313
5           2020        778
5           2021        1471
5           2022        787
6           2020        963
6           2021        1738
6           2022        826
7           2020        485
7           2021        2292
7           2022        1877
8           2020        923
8           2021        2843
8           2022        2147
9           2020        575
9           2021        955
9           2022        1140

(27 rows affected)

```

In [9]:
USE TSQLV4;
SELECT
  O.empid,
  YEAR(O.orderdate) AS orderyear,
  SUM(OD.qty) AS qty
FROM Sales.Orders AS O
  INNER JOIN Sales.OrderDetails AS OD
    ON O.orderid = OD.orderid
GROUP BY
  O.empid,
  YEAR(O.orderdate);


USE Northwinds2022TSQLV7;
SELECT
  O.EmployeeId,
  YEAR(O.OrderDate) AS orderyear,
  SUM(OD.Quantity) AS qty
FROM Sales.[Order] AS O
  INNER JOIN Sales.OrderDetail AS OD
    ON O.OrderId = OD.OrderId
GROUP BY
  O.EmployeeId,
  YEAR(O.OrderDate);



(27 rows affected)

(27 rows affected)

Total execution time: 00:00:00.052

empid,orderyear,qty
1,2014,1620
2,2014,1085
3,2014,940
4,2014,2212
5,2014,778
6,2014,963
7,2014,485
8,2014,923
9,2014,575
1,2015,3877


EmployeeId,orderyear,qty
1,2014,1620
2,2015,2604
6,2014,963
3,2016,2476
8,2014,923
5,2016,787
9,2015,955
7,2016,1877
3,2014,940
4,2015,5273
